##Logistic Regression - Python Demonstration
**This script will run a logistic regression on our demo dataset. The goal is to predict whether a customer will buy a subscription to a magazine based on a number of factors.**

###Steps:
1. Import necessary packages
    - We need the entire `pandas` and 'statsmodels.api' packages, but only specific *names* from the `math` and `sklearn` modules
    - Imports can be confusing to python beginners, for some additional details [see here](http://stackoverflow.com/a/21547572)
2. Read in the data sets
    - Also generate some summary statistics for info and data validation
    - Add in an 'intercept' column as a constant
3. Split our data into training and testing data sets using the `StratifiedShuffleSplit` from `sklearn`
4. Perform a logistic regression
    - Review results
    - Structure results into a readable format to view significant variables and interpret coefficients
5. Predict the outcome in the test data set and validate the model.

---

###1. Import necessary packages

In [4]:
# import relevant packages

import pandas as pd # for basic data frame functionality
import statsmodels.api as sm # for statistical modeling functionality with R-like syntax 

from math import exp # for exponentiating (not natively implemented in python)
from sklearn.cross_validation import StratifiedShuffleSplit # to form stratified train and test data

###2. Read in the data sets

In [5]:
data = pd.read_excel('LogRegData.xlsx') # using the read_excel function from pandas

- Generate summary statistics: What proportion of our dataset bought a subscription? This will give us a baseline for our model.

**What's the code doing?** Using the 'Buy' column from `data`, perform the `value_counts` function on it. `value_counts` has a `normalize` parameter which outputs the percentages rather than raw counts of the aggregation.

In [6]:
data['Buy'].value_counts(normalize=True)

0    0.814264
1    0.185736
dtype: float64

###2b. Add an intercept to our data set
For some reason, the models in the `statsmodels` package don't automatically add an intercept. So we have to add a column with a constant value to account for the intercept in the model.

In [7]:
data['(Intercept)'] = 1.0

### 3. Split our data into training and testing data sets using the `StratifiedShuffleSplit` from `sklearn`
**Note:** Declaring objects before using them is an odd concept for python beginners. What we're going to first do is define `sss` as a `StratifiedShuffleSplit` class. A class is an object that has associated functions or objects within it. The `StratifiedShuffleSplit` class performs a split that retains the stratified proportions of the target variable. It returns an index of values from the data frame that we then have to subselect. Essentially it's telling us how to cut the data set, not doing the actual cutting.

In [8]:
sss = StratifiedShuffleSplit(data['Buy'], n_iter=1, test_size=0.25, random_state=333)

Now we need to split the dataset using the results from our `StratifiedShuffleSplit`.

In [9]:
for train_index, test_index in sss:
    dataTrain, dataTest = data.ix[train_index], data.ix[test_index]

###4. Perform logistic regression
First we need to define our dependent and independent variables. We're going to define a list of our predictor columns by using python's [awesome list indexing](http://effbot.org/zone/python-list.htm). If we have a list of items (like columns in our dataset), we can select all columns from the first column to the end by appending it with `[1:]`.

In [10]:
predictorCols = data.columns[1:]

Now we declare a logit object as a Logit model from the statsmodels package. Since we called in the entire statsmodels package as `sm`, we then ust the Logit class from the `sm` package. The Logit class takes two required arguments: your target variable, and your predictor variables.

In [11]:
model = sm.Logit(dataTrain['Buy'], dataTrain[predictorCols])

After declaring the model, we need to also apply the `fit` method to the model since we haven't told the Logit class to *do* anything yet, and store the fitted model as a new object.

In [12]:
logitResults = model.fit()

Optimization terminated successfully.
         Current function value: 0.145078
         Iterations 10


The output tells us that the model converged after 10 iterations. We can now print out a summary of the model statistics.

**Note**: We can also use the .summary() function, but summary2() provides the AIC and BIC in the statistics table for us. These are available otherwise (we would just write `logitResults.aic()`, but it's nice to include them automatically).

In [13]:
logitResults.summary2()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                          Results: Logit
===================================================================
Model:                Logit            Pseudo R-squared: 0.698     
Dependent Variable:   Buy              AIC:              180.2388  
Date:                 2015-07-13 13:34 BIC:              252.0226  
No. Observations:     504              Log-Likelihood:   -73.119   
Df Model:             16               LL-Null:          -242.48   
Df Residuals:         487              LLR p-value:      2.3085e-62
Converged:            1.0000           Scale:            1.0000    
No. Iterations:       10.0000                                      
-------------------------------------------------------------------
                  Coef.   Std.Err.    z    P>|z|   [0.025   0.975] 
-------------------------------------------------------------------
Income             0.0002   0.0000  7.6556 0.0000   0.0001   0.0002
Female             1.6206   0.4977  3.2561 0.0011   0.6451   2.5961
Married            0.6135   0.6431  0.9539 0.3401  -0.6470   1.8739
Student           -0.1021   0.5019 -0.2033 0.8389  -1.0859   0.8817
Professional       0.0328   0.5373  0.0610 0.9514  -1.0203   1.0858
Retired           -1.2590   1.0369 -1.2142 0.2247  -3.2912   0.7732
Unemployed         0.9117   3.8095  0.2393 0.8109  -6.5548   8.3782
Residence Length   0.0211   0.0156  1.3530 0.1761  -0.0095   0.0516
Dual Income        0.3512   0.5880  0.5973 0.5503  -0.8013   1.5036
Minors             0.6606   0.5094  1.2970 0.1946  -0.3377   1.6590
Apartment          1.0709   0.6101  1.7553 0.0792  -0.1249   2.2667
Rent              -0.7703   0.6792 -1.1343 0.2567  -2.1015   0.5608
Own                1.5089   0.5944  2.5386 0.0111   0.3439   2.6739
English            1.7863   0.9208  1.9400 0.0524  -0.0184   3.5911
Prev Child Mag     1.8793   0.7817  2.4042 0.0162   0.3472   3.4114
Prev Parent Mag   -0.1584   0.7067 -0.2242 0.8226  -1.5434   1.2266
(Intercept)      -16.6282   2.2535 -7.3789 0.0000 -21.0449 -12.2114
===================================================================

"""

**Refine results**

We're going to use the `zip` function to tie all of our data together. The zip function takes group of equal length lists (vectors) and lines them up so they can be put into a DataFrame. The colums we're going to create are:
- **(Variable Names)** `predictorCols` -- We defined this up above, it's just the column names of our predictors
- **(Parameter Estimates)** `logitResults.params` -- This is the `params` attribute (parameters) of our logitResults object.
- **(Odds Ratios)** `(logitResults.params).apply(lambda x: round(exp(x), 4))` -- This one is complicated. We're going to take the parameters and apply a function to each of them. That function exponetiates the value to give us the odds ratio. Then we're going to round that number to 4 decimal digits.
- **(p-values)** `logitResults.pvalues.round(4)` -- the p-value of each of our variables
- **(significance)**  `logitResults.pvalues < 0.05` -- a binary indicator of whether our variable is significant at alpha = 0.05

In [14]:
resultColumns = zip(predictorCols, # gather out all of our predictor columns
                    logitResults.params, # gather the parameters 
                    (logitResults.params).apply(lambda x: round(exp(x), 4)), # gather the parameters, exponentiate, and round them
                    logitResults.pvalues.round(4), # gather the p-values, round them
                    logitResults.pvalues < 0.05) # print whether each variable is significant at alpha < 0.05

Now we put our data into a DataFrame for formatting.

In [15]:
resultsData = pd.DataFrame(resultColumns, columns=['Variable', 'Paremeter', 'Odds Ratio', 'Pr(>|z|)', 'Sig'])
resultsData

,Variable,Paremeter,Odds Ratio,Pr(>|z|),Sig
0,Income,0.000184,1.0002,0.0000,True
1,Female,1.620632,5.0563,0.0011,True
2,Married,0.613478,1.8468,0.3401,False
3,Student,-0.102066,0.9030,0.8389,False
4,Professional,0.032768,1.0333,0.9514,False
5,Retired,-1.258998,0.2839,0.2247,False
6,Unemployed,0.911673,2.4885,0.8109,False
7,Residence Length,0.021071,1.0213,0.1761,False
8,Dual Income,0.351180,1.4207,0.5503,False
9,Minors,0.660632,1.9360,0.1946,False


###5. Predict the outcome in the test data set and validate the model.
Now that we have a model we're going to validate it's accuracy against a test data set. Our model object `logitResults` has a  'predict' function that generates predicted probabilities for a set of observations with the same independent variables. We're going to store these probabilities, then add them as a column to our `dataTest` data set.

In [29]:
predictions = logitResults.predict(dataTest[predictorCols])

In [30]:
dataTest['Prediction Prob'] = predictions

- Since we have predicted probabilities we need to determine a cutoff. Before this step, you would need to perform some cutoff calibration using something like [ROC Curves](http://blog.yhathq.com/posts/roc-curves.html) or [other methods](http://scikit-learn.org/stable/modules/calibration.html). For our purposes, we'll assume a probability above 0.5 is a "True".

In [31]:
dataTest['Prediction'] = dataTest['Prediction Prob'] > 0.5

- A common method of verifying model accuracy is using a [confusion matrix](https://en.wikipedia.org/wiki/Confusion_matrix). We'll use `pandas' crosstab` function to create a nice one. 

In [33]:
pd.crosstab(dataTest['Buy'], dataTest['Prediction'], rownames=['Actual'], colnames=['Predicted'], margins=True)

Predicted,False,True,All
Actual,,,
0,136,2,138
1,6,25,31
All,142,27,169


There are many different metrics used to evaluate a model, but True Positives, True Negatives, and Accuracy are good basic ones. We can calculate them all using the confusion matrix numbers and some basic calculations.

In [40]:
from __future__ import division

######re: above
We're importing division as it's fixed in Python 3. Python 2 uses integer division by default. Don't worry about it, it's complicated. If you ever get weird division results, this is why. https://www.python.org/dev/peps/pep-0238/

In [38]:
true_pos = 25/31
true_neg = 136/138
accuracy = (136 + 25) / 169

print "True Positives: " + str(round(true_pos, 4))
print "True Negatives: " + str(round(true_neg, 4))
print "Accuracy: " + str(round(accuracy, 4))

True Positives: 0.8065
True Negatives: 0.9855
Accuracy: 0.9527


###Recap
We've created and fit a predictive model, developed a readable output for the coefficients, and used a model to predict additional observations for model validation.

**Extra Credit** (some ideas for improvement):
- Run a different random seed into the shuffle to verify model results. Do significant variables stay consistent despite the seed?
- Check out some other [model evaluation metrics](http://scikit-learn.org/stable/modules/model_evaluation.html#classification-metrics).
- Our model includes all of our original variables, despite their significance. Testing whether a simplified model would give us similar results without a huge loss in accuracy would be beneficial.
- Repeat the modeling using [`sklearn's LogisticRegression`](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html). What's different?